# Semantic Kernel with OpenBnB MCP Server Integration

ဤနိုက်ဘုတ်တွင် Semantic Kernel ကို အသုံးပြု၍ OpenBnB MCP ဆာဗာနှင့် ပေါင်းစည်းပြီး MCPStdioPlugin ကို အသုံးပြုကာ အမှန်တကယ် Airbnb တည်းခိုခန်းများ ရှာဖွေခြင်းကို ပြသထားသည်။ LLM Access အတွက် Microsoft Foundry ကို အသုံးပြုသည်။ သင့်ပတ်ဝန်းကျင်အပြောင်းအလဲများကို စီမံရန် [Setup Lesson](/00-course-setup/README.md) ကို လိုက်နာနိုင်ပါသည်။


## လိုအပ်သောပက်ကေ့ဂျ်များကိုတင်သွင်းခြင်း


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP Plugin ချိတ်ဆက်ခြင်း ပြုလုပ်ခြင်း

ကျွန်ုပ်တို့သည် MCPStdioPlugin ကို အသုံးပြု၍ [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) တွင် ချိတ်ဆက်မည်ဖြစ်သည်။ ဤ server သည် @openbnb/mcp-server-airbnb package ဖြင့် Airbnb ရှာဖွေရေး လုပ်ဆောင်ချက်ကို ပံ့ပိုးပေးသည်။


## Client ဖန်တီးခြင်း

ဤနမူနာတွင် ကျွန်ုပ်တို့သည် LLM ဝင်ရောက်မှုအတွက် Microsoft Foundry ကို အသုံးပြုမည်ဖြစ်သည်။ သင့်ပတ်ဝန်းကျင် အပြောင်းအလဲများကို မှန်ကန်စွာ ဆက်တင်ထားကြောင်း သေချာအောင် ပြုလုပ်ပါ။


## ပတ်ဝန်းကျင် ဆက်တင်များ

Azure OpenAI ဆက်တင်များကို ပြင်ဆင်ပါ။ အောက်ပါ ပတ်ဝန်းကျင် အပြောင်းအလဲများကို သတ်မှတ်ထားကြောင်း သေချာစေပါ-
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ဖက်ဆင်မှုကိုနားလည်ခြင်း

ဤနောက်စာမျက်နှာသည် **အမှန်တကယ် OpenBnB MCP ဆာဗာ** နှင့်ချိတ်ဆက်ပြီး Airbnb ရှာဖွေရေးလုပ်ဆောင်ချက်ကိုပေးသည်။

### ၎င်းသည် ဘယ်လိုလုပ်ဆောင်သနည်း:

1. **MCPStdioPlugin**: MCP ဆာဗာနှင့် စံပုံရိပ် အထွက်/အဝင် ဆက်သွယ်မှုကို အသုံးပြုသည်
2. **အမှန်တကယ် NPM Package**: npx ရှိပြီးဖြင့် `@openbnb/mcp-server-airbnb` ကိုဒေါင်းလုပ်လုပ်နှင့် စတင်ရိုက်ထည့်သည်
3. **၀ါ live Data**: ၎င်းတို့၏ API များမှ အမှန်တကယ် Airbnb အိမ်ရာဒေတာကို ပြန်သွားပေးသည်
4. **လုပ်ဆောင်ချက်ရှာဖွေခြင်း**: အေးဂျင့်သည် MCP ဆာဗာမှ အသုံးပြုနိုင်သော လုပ်ဆောင်ချက်များကို အလိုအလျောက် ရှာဖွေသည်

### ရရှိနိုင်သော လုပ်ဆောင်ချက်များ:

OpenBnB MCP ဆာဗာသည် အများအားဖြင့် ပေးထားသည်မှာ-
- **search_listings** - တည်နေရာနှင့် မူဝါဒများအရ Airbnb အိမ်ရာများကို ရှာဖွေရန်
- **get_listing_details** - အထူးသတ်မှတ်သော အိမ်ရာများအကြောင်း အသေးစိတ် အချက်အလက်များ ရယူရန်
- **check_availability** - သတ်မှတ်ရက်စွဲများအတွက် ရရှိနိုင်မှု စစ်ဆေးရန်
- **get_reviews** - အိမ်ရာများအတွက် သုံးသပ်ချက်များ ရယူရန်
- **get_host_info** - အိမ်ရာပိုင်ရှင်ဆိုင်ရာ အချက်အလက်များ ရယူရန်

### လိုအပ်ချက်များ:

- သင့်စက်တွင် **Node.js** ထည့်သွင်းထားရန်
- MCP ဆာဗာ package ကိုဒေါင်းလုပ်လုပ်ရန် အင်တာနက် ချိတ်ဆက်မှုရှိရန်
- **NPX** ရရှိနိုင်ရန် (Node.js နှင့်အတူပါဝင်သည်)

### ချိတ်ဆက်မှု စမ်းသပ်ခြင်း:

အောက်ပါရေးသားချက်အား လက်ဖြင့် တစ်ဆင့်စမ်းသပ်နိုင်ပါသည်-
```bash
npx -y @openbnb/mcp-server-airbnb
```

ဤသည်က OpenBnB MCP ဆာဗာကို ဒေါင်းလုပ်လုပ်ပြီး စတင်ကာ Semantic Kernel မှ အသုံးပြု၍ အမှန် Airbnb ဒေတာများကို ချိတ်ဆက်သည့် လမ်းကြောင်း ဖြစ်သည်။


## OpenBnB MCP Server နှင့် AI Agent ကို ပြေးဆွဲခြင်း

ယခုတွင် Stockholm အတွက် လူကြီး ၂ ဦး နှင့် ကလေး ၁ ဦးအတွက် အမှန်တကယ် Airbnb တည်းခိုးစခန်းများကို ရှာဖွေသည့် OpenBnB MCP server နှင့် ချိတ်ဆက်ထားသော AI Agent ကို ပြေးဆွဲပါမည်။ ရှာဖွေရေးအချက်အလက်များကို ပြင်ဆင်လိုပါက `user_inputs` စာရင်းကို ပြောင်းလဲနိုင်ပါသည်။


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# အကျဉ်းချုပ်
အောင်မြင်ပါပြီ! သင်သည် Model Context Protocol (MCP) ကို အသုံးပြုပြီး အရ πραγμα accommodation ရှာဖွေရေးနှင့် ပေါင်းစည်းထားသော AI ကိုယ်စားလှယ်ကို အောင်မြင်စွာ တည်ဆောက်ပြီး ဖြစ်ပါသည်။

## အသုံးပြုထားသောနည်းပညာများ
- Semantic Kernel - Azure OpenAI ဖြင့် အတတ်ပညာရှင်ကိုယ်စားလှယ်များ တည်ဆောက်ရန်
- Microsoft Foundry - LLM စွမ်းဆောင်ရည်များနှင့် စကားပြောပြီးစီးမှုအတွက်
- MCP (Model Context Protocol) - စံနှုန်းထားအင်တဂရိတ်မှုအတွက်
- OpenBnB MCP Server - အမှန်တကယ် Airbnb ရှာဖွေရေးလုပ်ဆောင်ချက်အတွက်
- Node.js/NPX - ပြင်ပ MCP server ကို အလိုအလျောက် စတင်ရန်

## သင်သိရှိခဲ့သောအရာများ
- MCP ပေါင်းစည်းခြင်း - Semantic Kernel ကိုယ်စားလှယ်များကို ပြင်ပ MCP server များနှင့် ချိတ်ဆက်ခြင်း
- အချိန်နှင့်တပြေးညီ ဒေတာ ဝင်ရန် - တိကျသော Airbnb အိမ်ယာများကို တိုက်ရိုက် API များမှရှာဖွေခြင်း
- ပရိုတိုကော ဆက်သွယ်မှု - ကိုယ်စားလှယ်နှင့် MCP server အကြား stdio ဆက်သွယ်မှု အသုံးပြုခြင်း
- အလုပ်အကိုင်ရှာဖွေခြင်း - MCP server များမှ ရရှိနိုင်သောလုပ်ဆောင်ချက်များကို အလိုအလျောက် ရှာဖွေရန်
- ပုံနှိပ်ရုပ်ပုံ တုံ့ပြန်ချက်များ - လုပ်ဆောင်ချက်ခေါ်ဆိုမှုများကို တိုက်ရိုက်ဖမ်းယူပြီး မှတ်တမ်းတင်ခြင်း
- HTML ပြသမှု - တိုက်ရိုက်စာရင်းကို စတိုင်ဖြင့် ဖော်ပြ၍ ပေါ်လွင်သော ပြသမှုများ ဖန်တီးခြင်း

## နောက်တခါဆက်လုပ်ရန်
- ပိုပြီး MCP server များ (ရာသီဥတု၊ ငွေကြေး၊စားသောက်ဆိုင်များ) ပေါင်းစည်းရန်
- MCP နှင့် A2A ပရိုတိုကောများ ပေါင်းစပ်သော မိုက်တီကိုယ်စားလှယ်စနစ် တည်ဆောက်ရန်
- သင့်ကိုယ်ပိုင် ဒေတာရင်းမြစ်များအတွက် စိတ်ကြိုက် MCP server များ ဖန်တီးရန်
- အတူတကွ စကားပြောသမိုင်းများ ထိန်းသိမ်းရေး တည်ဆောက်ရန်
- MCP server များစွာကို အုပ်ချုပ်သည့်အဖြစ် Azure Functions တွင် ကိုယ်စားလှယ် တင်ဆောင်ရန်
- အသုံးပြုသူ အတည်ပြုခြင်းနှင့် စာရင်းသွင်းမှု စွမ်းရည်များ ပေါင်းထည့်ရန်


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**သတိပေးချက်**  
ဤစာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ဖြင့် ဘာသာပြန်ထားခြင်းဖြစ်ပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးပမ်းဆောင်ရွက်နေသော်လည်း၊ အလိုအလျောက်ဘာသာပြန်ခြင်းအထဲတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများဖြစ်ပေါ်နိုင်ပါကြောင်း သိရှိထားရန်လိုအပ်ပါသည်။ အစစ်အမှန်ရှိသော အမျိုးဘာသာဖြင့် ရေးသားထားသော မူလစာတမ်းကို အာဏာပိုင်အရင်းအမြစ်အဖြစ် သတ်မှတ်ရမည်ဖြစ်သည်။ အရေးကြီးသော အချက်အလက်များအတွက် မူကြောင်းလူသားဘာသာပြန်ချက်ကို အကြံပြုပါသည်။ ဤဘာသာပြန်ချက် အသုံးပြုမှုကြောင့် ဖြစ်ပေါ်လာနိုင်သည့် နားလည်မှားယွင်းခြင်းများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မခံပါ။
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
